# Soft Max

softmax는 binary classification을 여러 번 결합한 결과다. 예측 결과가 A, B, C 중의 하나가 되어야 한다면, 동일한 x에 대해 A가 될 확률, B가 될 확률, C가 될 확률을 모두 구해야 한다. x는 3번 사용되지만, W는 A, B, C에 대해서 한 번씩 필요하니까 3번 반복된다. 그래서, W는 예측해야 하는 숫자만큼 필요하게 된다. 10개 중에서 예측한다면 10개의 W가 나와야 한다.

$$
\mathbf{S(Y_i)} = \frac{e^y_i}{\sum\limits_{j=1}^{m}{e^y_j}}
$$

# One-Hot Encoding

ML에서 결과로 보통 두개의 근접한 값들이 거리가 있는 두 개의 값들보다 더 유사하다고 가정하는데, 
예를 들면 결과값이 1인 경우 3보다는 2와 비슷하다고 여겨지는 것을 말합니다.

그런데 ML 예측의 결과로 A, B, C 같이 전혀 관련성이 없는 분류들 중에 하나를 결과로 내놓는 경우는
결과값이 숫자이기 보다는 A, B, C 같은 결과로 도출되길 원할 경우 원-핫 인코딩 기법을 사용합니다.

설명은 장황하지만, 간단히 요약하자면 메모리에서 하나의 비트들을 각 결과로 매칭시켜 A와 유사할 경우
A에 해당하는 bit를 1, 아닐 경우 0 같은 방식으로 처리하는 것을 말합니다

예측결과 A와 유사 = 100(A__)<br>
예측결과 A, C의 속성을 갖음 = 101(A_C)<br>
예측결과 B, C의 속성을 갖음 = 011(_BC)<br>
예측결과 모든 속성을 갖음 = 111(ABC)<br>

# Cross-Entropy Cost Function

$$
[S(=y): softmax값,\ L(=\bar{y}): 0\ or\ 1]\\
D(S, L) = -\sum\limits_i L_{i}\log{(S_i)} \\
Cost = \frac{1}{N} \sum_{i=1}^N {D(S(wx_i + b), L_i)}
$$